# 네이버 영화리뷰 감성분석 : 나이브베이지안 (sklearn)

In [58]:
import pandas as pd
import numpy as np

In [90]:
df = pd.DataFrame(pd.read_csv('C:/Users/ASIA/Downloads/nsmc-master/nsmc-master/ratings_train.txt', sep='\t', quoting=3))
df = df.replace(np.nan , '', regex = True)
df = df[:10000]
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 5점주려고했는데 1 한국 공포영화의 특징인 깜놀시키려 하...,0
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,0
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음.,1
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버11씨는 살아남길 바랬는데 2번째극장판 어서 나오길,1


In [91]:
# df의 document 부분을 분석하려고 하므로, 편리하게 하기 위해
import re
# 한글을  제외한 다른 문자를 제거하자
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")  #이 표현은 그냥 한글

def preprocess(text) :
    text = re.sub(remove_except_ko,' ', text).strip()
    return text

df['document'] = df['document'].map(lambda x : preprocess(x))



In [92]:
df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 점주려고했는데 한국 공포영화의 특징인 깜놀시키려 하...,0
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,0
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음,1
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버 씨는 살아남길 바랬는데 번째극장판 어서 나오길,1


In [93]:
df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 점주려고했는데 한국 공포영화의 특징인 깜놀시키려 하...,0
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,0
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음,1
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버 씨는 살아남길 바랬는데 번째극장판 어서 나오길,1


In [94]:
import konlpy.tag

In [95]:
from konlpy.tag import Mecab
mecab = Mecab()

#mecab으로 토큰화하고 한 글자는 젝하고, stop_word도 제거하자

stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']

# 이거는 한글자 빼고 / stopword 랑 stopPos 뺴는거. 이따가 쓸 것임. text는 꼭 str
def token_mecab(text) :
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])> 1]
    text = [i for i in text if i[0] not in stop_word] 
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

# 이런식으로 짜보려고 시도 했으나.... 더 편리한 방법들이 있었던 것!
# for i in range(len(X_train)) :
#     a = str(X_train[i])
#     a = a.replace('[','')
#     a = a.replace(']', '')
#     a = a.replace("'",'')
#     a = a.replace(',','')
#     X_train[i] = a



In [96]:
# 토큰화 + 토큰리스트 생성
def make_tokens(df) :
    df['tokens'] = ''
    tokens_list = []
    for i, row in df.iterrows() :
        if i %10000 == 0 :
            print(i, '/', len(df))
        token = token_mecab(df['document'][i])
        df['tokens'][i] = ' '.join(token)
        
    return df

df = make_tokens(df)

0 / 10000


C:\Users\ASIA\Anaconda3\envs\sjpark_environment\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [97]:
df
# 이런 식으로 이제 document 를 전처리한 tokens가 나옴!!1 그럼 tokens와 label 가지고 돌리기 시작!

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,짜증 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,포스터 초딩 영화 오버 연기 가볍 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,교도소 이야기 구먼 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 보이 커스틴 던스트 이뻐 보였
...,...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 점주려고했는데 한국 공포영화의 특징인 깜놀시키려 하...,0,티비 무료 때문 점주 려고 는데 한국 공포 영화 특징 깜놀 시키 장면 깜놀 재미없 ...
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,0,드라마 라고 수습 으면 걍친자녀아니면되고 간단 얼굴 바꿨으면 결말 라도 정리 던가 ...
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음,1,김연아 에리카
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버 씨는 살아남길 바랬는데 번째극장판 어서 나오길,1,ㅋㅋ ㅋㅋ 넘버 살아남 바랬 는데 극장판 나오


In [105]:
# 트레인!
X_train = list(df['tokens'])
Y_train = list(df['label'])

# 이거 아님 주의 !!
# X_train = list(df['document'])
# Y_train = list(df['label'])

In [106]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [107]:
# 예측
print(clf.predict(count_vect.transform(["문신처럼 새긴 인간의 불완전성!"]))) 
# 확률 
print(clf.predict_proba(count_vect.transform(["문신처럼 새긴 인간의 불완전성!"])))

[0]
[[0.5021 0.4979]]


In [108]:
text = '2020년 5월 8일은 꿈만같았던 날이었다 '

In [109]:
def check(text) :
    print(clf.predict(count_vect.transform([text]))) 
    print(clf.predict_proba(count_vect.transform([text])))

In [110]:
check(text)

[0]
[[0.5021 0.4979]]


# 예측결과 정확도를 측정하자

In [111]:
# 테스트!
import pandas as pd 
test_df = pd.DataFrame(pd.read_csv('C:/Users/ASIA/Downloads/nsmc-master/nsmc-master/ratings_test.txt', sep='\t', quoting=3))
test_df = test_df.replace(np.nan, '', regex=True)
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
test_df['document'] = test_df['document'].map(lambda x : preprocess(x))

In [112]:
test_df = make_tokens(test_df)
test_df.head()

0 / 50000


C:\Users\ASIA\Anaconda3\envs\sjpark_environment\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 50000
20000 / 50000
30000 / 50000
40000 / 50000


,id,document,label,tokens
0,6270596,굳 ㅋ,1,
1,9274899,GDNTOPCLASSINTHECLUB,0,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루 은데 완전 막장
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,아니 어도 다섯 텐데 나와서 심기 불편


In [122]:
# predic_label은 실제로 예측한것!!!!!
predic_label = []
for i, row in test_df.iterrows():
    predic = clf.predict(count_vect.transform([test_df['tokens'][i]]))[0]
    predic_label.append(predic)
test_df['predic_label'] = predic_label
test_df.head(10)

,id,document,label,tokens,predic_label
0,6270596,굳 ㅋ,1,,0
1,9274899,GDNTOPCLASSINTHECLUB,0,,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루 은데 완전 막장,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,아니 어도 다섯 텐데 나와서 심기 불편,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1,음악 최고 음악 영화,1
6,6315043,진정한 쓰레기,0,진정 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0,미국 애니 튀어나온 창의 로봇 디자인 고개 한다,1
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0,개판 중국 영화 유치 내용 무기 유치 남무 그립 동사서독 영화 이건 아류,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1,이별 아픔 찾아오 새로운 인연 기쁨 사람 그렇,1


## 이런식으로 confusion matrix를 만들어봄

- confusion metrix
- 표	참	거짓
- 참	TP	FP
- 거짓	FN	TN




In [123]:
# 계산하는 방법임
# predic 과 실제 label을 비교하여 정의대로 값을 뽑아낸다.
TP = FN = FP = TN = other = 0
for i in range(len(test_df)):
    if test_df['label'][i] == 1 and test_df['predic_label'][i] == 1:
        TP +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 0:
        TN +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 1:
        FN +=1
    elif test_df['label'][i] == 1 and test_df['predic_label'][i] == 0:
        FP +=1
    else:
        other +=1
print(TP,TN,FN,FP)

19162 19497 5330 6011


In [124]:
ACC = (TP+TN)/(TP+TN+FP+FN)
print('ACC: ',ACC)

ACC:  0.77318
